web retriever

In [ ]:
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

# langsmith api key load
load_dotenv()

# 적당한 예시 기사
news_url = "https://www.chosun.com/international/international_general/2024/11/18/KHYABRBY5JEO7EPSR32CVRFXXA/"

# 한국어 파인튜닝 ollama 모델
llm = ChatOllama(model="EEVE-Korean-10.8B:latest")

In [ ]:
import bs4
from langchain.chains import create_retrieval_chain
from langchain.tools.retriever import create_retriever_tool
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 어떤 페이지를 가져올지 명시
loader = WebBaseLoader(
    web_paths=(news_url,),
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("article-header__headline", "article-body") 가져올 html tag 정보, 안주면 모두 가져옴
    #     )
    # )
)

# 페이지 정보 가져오기
docs = loader.load()

# 적당한 크기로 잘라주는 splitter 생성
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)

# splitter 로 페이지 자르기
splits = text_splitter.split_documents(docs)

# 잘라진 페이지 임베딩해서 스토리지에 저장하기
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits,
    embedding=OllamaEmbeddings(model="bge-m3")
)

# 리트리버 객체 생성
retriever = vectorstore.as_retriever()

# 시스템 프롬프트 (여기를 이것 저것 시도해 봐야함)
system_prompt = (
    # (대충 아는 것만 대답하라는 뜻)
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

# 입력 시 프롬프트 
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# 프롬프트 후 llm 사용하는 객채
question_answer_chain = create_stuff_documents_chain(
    llm, prompt
)

# 리트리버를 참조하는 객체
rag_chain = create_retrieval_chain(
    retriever, question_answer_chain
)

In [ ]:
response = rag_chain.invoke({
    # 뉴스 내용 질문
    "input": "누가 햄버거를 독극물이라고 했지?"
})
response["answer"]

Agent 생성

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

# 충분히 지능이 높은 모델
model = ChatOllama(model="llama3.1:70b")


# llm이 사용할 수 있게끔 tool 객체화
retriever_tool = create_retriever_tool(
    retriever,
    "news_search",
    "트럼프 관련 뉴스를 검색한다." # tool에 기능 설명 (이 부분을 자세히 할 수록 적절한 답변이 나옴)
)

# agent 에게 넘길 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. received from users into Korean."),
        # ("system", "You are a translator that translates sentences received from users into Korean."),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# tool 리스트
tools = [retriever_tool]

# 어떤 툴을 사용할 수 있는지 알려줌
agent = create_tool_calling_agent(model, tools, prompt)

# 실제로 툴을 사용할 수 있게 해줌
agent_executer = AgentExecutor(agent=agent, tools=tools)

In [ ]:
agent_executer.invoke({"input": "트럼프 뉴스 뭐 있어?"}) # 알아서 tool을 사용하는지 확인